**INITIALISATION** 

In [2]:
# Librairies 
from easytello import tello
from darkflow.net.build import TFNet
import cv2
import numpy as np
import math

# Parametres du modele
options = {"model" : "yolo/cfg/yolo-lite.cfg", "load": "yolo/weights/yolo-lite.weights", "threshold" : 0.26}
# Initialisaton du modele
yolo = TFNet(options)

Parsing yolo/cfg/yolo-lite.cfg
Loading yolo/weights/yolo-lite.weights ...
Successfully identified 2597684 bytes
Finished in 0.0019905567169189453s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 224, 224, 3)
 Load  |  Yep!  | conv 3x3p1_1    leaky            | (?, 224, 224, 16)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 112, 112, 16)
 Load  |  Yep!  | conv 3x3p1_1    leaky            | (?, 112, 112, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 56, 56, 32)
 Load  |  Yep!  | conv 3x3p1_1    leaky            | (?, 56, 56, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 28, 28, 64)
 Load  |  Yep!  | conv 3x3p1_1    leaky            | (?, 28, 28, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 14, 14, 128)
 Load  |  Yep!  | conv 3x3p1_1    leaky            | (?, 14, 14, 128)
 L

In [3]:
# Objet Tello() 
my_drone = tello.Tello()

OSError: [Errno 98] Address already in use

**VIDEO**

In [4]:
# Il faut d'abord demander au drone de nous fournir son stream video grace a la fonction streamon()

my_drone.streamon()

Sending command: command
Connection timed out!
Response: None
Sending command: streamon
Connection timed out!
Response: None


In [ ]:
# Importation d'outils yolotools

from yolotools import VideoCapture
from yolotools import bounding_boxes
from yolotools import detected

# Creation de l'objet capture 
# La source du video est la variable video_source de l'objet Tello() 

capture = VideoCapture(my_drone.video_source)

In [ ]:
# Une fois l'objet capture initialise, nous pouvons jouer le video comme dans le chapitre precedent. 

while drone.stream_state():
    
    frame = capture.read()
    
    if frame is None:
        break
    
    cv2.imshow("Tello Stream", frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

capture.release()
cv2.destroyAllWindows()  

**YOLO**

In [ ]:
# Comme dans le chapitre precedent, on peut modifier la boucle pour integrer YOLO. 

capture = VideoCapture(my_drone.video_source)

while drone.stream_state():
    
    frame = capture.read()
    
    if frame is None:
        break
    
    predictions = yolo.return_predict(frame)
    
    bounding_boxes(predictions, frame)
    
    cv2.imshow("Tello Stream", frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

capture.release()
cv2.destroyAllWindows()  

**YOLO POUR CONTROLE**

In [5]:
# Pour l'instant, Tello ne bouge pas. 
# On peut modifier la boucle pour le controler conditionnellement au contenu de la variable "predictions"
# La fontion detected(predictions, label) retourne True si un objet de la classe "label" est detecte dans les predictions 
# Si une personne est detectee - detected(predictions, "person") retourne True -, le drone fait une pirouette, affiche un message et sort de la boucle. 

capture = VideoCapture(my_drone.video_source)

my_drone.takeoff()
my_drone.up("100")

while drone.stream_state():
    
    frame = capture.read()
    
    if frame is None:
        break
    
    predictions = yolo.return_predict(frame)
    
    bounding_boxes(predictions, frame)
    
    cv2.imshow("Tello Stream", frame)
    
    # Si une "person" est presente dans les predictions, faire une pirouette, afficher un message a l'ecran, et sortir de la boucle. 
    
    if detected(predictions, "person"):
        my_drone.flip("left")
        print("Person detected!")
        break 
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

my_drone.land()
        
capture.release()
cv2.destroyAllWindows()  

my_drone.streamoff() 

NameError: name 'VideoCapture' is not defined